# Transfer Learning with RAG Models
----
**Objective**: In this notebook, you will experiment with a QA RAG model on the climate_fever dataset using the Haystack framework. You will get to go through the whole process of fitting the parts of a RAG model together, and learn how to prompt it with queries to get answers from the provided dataset.

NOTE: Make sure to change the runtime from CPU to TPU or GPU for faster training

## Install Libraries
Install the Haystack (for colab) and Datasets libraries

In [ ]:
!pip install farm-haystack[colab]
!pip install datasets

  Using cached huggingface_hub-0.17.3-py3-none-any.whl (295 kB)
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.19.4
    Uninstalling huggingface-hub-0.19.4:
      Successfully uninstalled huggingface-hub-0.19.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 2.15.0 requires huggingface-hub>=0.18.0, but you have huggingface-hub 0.17.3 which is incompatible.
  Using cached huggingface_hub-0.19.4-py3-none-any.whl (311 kB)
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.17.3
    Uninstalling huggingface-hub-0.17.3:
      Successfully uninstalled huggingface-hub-0.17.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tokenizers 0.14.1 requires hugg

## Import Dataset
----
In this section, we will use as an example the climate_fever dataset. The dataset consists of 1535 rows of claims about climate change, and they either refute or support climate change, with some claims being neutral. We will build with a specific topic in mind so that we can get more accurate answers, and keep in mind that bigger datasets with open topics can also be used.

**Question 1**: Use the "load_dataset" function to load the "climate_fever" dataset with the "test" split.

In [ ]:
from datasets import load_dataset

# Load the dataset
dataset = load_dataset("climate_fever" , split = "test")

Generating test split:   0%|          | 0/1535 [00:00<?, ? examples/s]

In [ ]:
print(dataset)

Dataset({
    features: ['claim_id', 'claim', 'claim_label', 'evidences'],
    num_rows: 1535
})


## Formatting and Writing Documents
----
First, we need to extract, format and write the documents from our chosen dataset so that we can later build our QA Pipeline. This Pipeline will facilitate the process of building our RAG model and getting answers from it.

Keep in mind that for this notebook we will focus on how to build the pipeline with the simplest configurations. Feel free to experiment with different parameters.



**Question 2**: Use the write_documents method to save the formatted documents into document_storage

In [ ]:
from haystack.document_stores import InMemoryDocumentStore

# Extract and format the documents from the dataset
formatted_documents = [{"content": doc["claim"]} for doc in dataset]


document_storage = InMemoryDocumentStore(use_bm25=True)

# Write the documents to the document store
document_storage.write_documents(formatted_documents)


Updating BM25 representation...: 100%|██████████| 1535/1535 [00:00<00:00, 107043.80 docs/s]


In [ ]:
print(formatted_documents)

[{'content': 'Global warming is driving polar bears toward extinction'}, {'content': 'The sun has gone into ‘lockdown’ which could cause freezing weather, earthquakes and famine, say scientists'}, {'content': 'The polar bear population has been growing.'}, {'content': "Ironic' study finds more CO2 has slightly cooled the planet"}, {'content': 'Human additions of CO2 are in the margin of error of current measurements and the gradual increase in CO2 is mainly from oceans degassing as the planet slowly emerges from the last ice age.'}, {'content': 'They tell us that we are the primary forces controlling earth temperatures by the burning of fossil fuels and releasing their carbon dioxide.'}, {'content': 'The Great Barrier Reef is experiencing the most widespread bleaching ever recorded'}, {'content': 'it’s not a pollutant that threatens human civilization.'}, {'content': 'If CO2 was so terrible for the planet, then installing a CO2 generator in a greenhouse would kill the plants.'}, {'cont

## Preparing the Retriever
----
We need to prepare our Retriever node of our pipeline. It will be responsible to get the documents from our document storage, so that they can be used by the Language Model later. We will the BM25Retriever provided by haystack, as it is the recommended Retriever for begginners.

**Question 3**: Create the BM25Retriever using the document_storage created earlier, with a top_k of value 2

In [ ]:
from haystack.nodes import BM25Retriever

# Note: The higher the top_k is, the better the answer will be. However, speed will be affected
retriever = BM25Retriever(document_store=document_storage, top_k=2)

## Preparing the Language Model
----
Now, we will prepare our Language Model using the prompt node. We need to first create our prompt, and for that, Haystack requires a specific structure. We will then define our desired language model alongside the prompt template we created. When creating this template, we need to Parse the output to a format that Haystack can use.

**Question 4**: Define the prompt node using PromptNode with the model name as "google/flan-t5-large" and the default prompt template as the created "rag_prompt"

In [ ]:
from haystack.nodes import PromptNode, PromptTemplate, AnswerParser

rag_prompt = PromptTemplate(
    prompt="""Create comprehensive answers from the related text given the questions.
                             Provide a clear and concise response that displays the key points and information presented in the related text.
                             Your answer should be in your own words and be no longer than 50 words.
                             \n\n Related text: {join(documents)} \n\n Question: {query} \n\n Answer:""",
    output_parser=AnswerParser(),
)

prompt_node = PromptNode(model_name_or_path="google/flan-t5-large", default_prompt_template=rag_prompt)

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

## Fitting our Pipeline Together
----
Finally, we are going to put our pipeline nodes together. For that we will use the Pipeline function from haystack. With the pipeline ready you will be able to ask it questions and get answers

**Question 5**: Add the retriever node and prompt_node created in the previous steps to the Pipeline using the add_node function. Hint: you need to provide the inputs to each of these nodes.

In [ ]:
from haystack.pipelines import Pipeline

pipe = Pipeline()

# Add the retriever node
pipe.add_node(component=retriever, name="Retriever", inputs=["Query"])
# Add the prompt node
pipe.add_node(component=prompt_node, name="PromptNode", inputs=["Retriever"])



## Asking the RAG Model Questions
----
We use the pipeline .run() method to ask a question. Since the output provided by our Prompt Node is a Haystack object, we retrieve in the way provided inside the print() function.

In [ ]:
output = pipe.run(query="When did global warming start")

print(output["answers"][0].answer)

The 20th century global warming did not start until 1910.


In [ ]:
# Here are some other examples you can use
examples = [
    "Who is most responsible for pollution",
    "What is the biggest damaging factor for the climate?",
    "What are some clean energy sources?",
    "How much does the average temperature of our planet rise per decade?"
]

In [ ]:
for example in examples:
    output = pipe.run(query=example)
    answer_text = output["answers"][0].answer
    print(f"Question: {example}\nAnswer: {answer_text}\n")

Question: Who is most responsible for pollution
Answer: The IPCC 95% confidence that humans are responsible for most of the current global warming is simply a summary of the peer-reviewed scientific research.

Question: What is the biggest damaging factor for the climate?
Answer: Burping cows are more damaging to the climate than all the cars on this planet.

Question: What are some clean energy sources?
Answer: Solar, wind and geothermal.

Question: How much does the average temperature of our planet rise per decade?
Answer: The average temperature of our planet has risen by around 1.62F (0.9C).

